In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

import os
import zlib
import zipfile

import matplotlib.pylab as plt
import matplotlib.image as mpimage
%matplotlib inline

In [0]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip
  

--2019-07-04 15:24:36--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   146MB/s    in 0.4s    

2019-07-04 15:24:37 (146 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ = zipfile.ZipFile(local_zip)
zip_.extractall('/tmp/catc_and_dogs_filtered')
zip_.close()

In [0]:
base_dir = '/tmp/catc_and_dogs_filtered/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
validation_cats_dir = os.path.join(validation_dir, 'cats')

train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')



In [0]:
def plot_images(item_dir, top = 25):
  all_items_dirs = os.listdir(item_dir)
  item_files = [os.path.join(item_dir,file) for file in all_items_dirs][:top]

  plt.figure(figsize =(10,10))
  for idx, img_path in enumerate(item_files):
    plt.subplot(5, 5, idx+1)

    img = mpimage.imread(img_path)
    plt.imshow(img)

  plt.plt.tight_layout()
  

In [0]:
#plot_images(train_dogs_dir)

In [0]:
input_shape = (150,150,3)

model = Sequential([
    Conv2D(32, (3,3), input_shape=input_shape),
    MaxPool2D((2,2)),
    
    Conv2D(64, (3,3), activation = 'relu'),
    MaxPool2D((2,2)),
    
    Conv2D(64, (3,3), activation = 'relu'),
    MaxPool2D((2,2)),

    Flatten(),
    
    Dense(64, activation = 'relu'),
    Dropout(0.5),
    Dense(1, activation = 'sigmoid')

])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 18496)             0         
__________

In [0]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size = (150, 150),
  batch_size = 20,
  class_mode = 'binary'
)

validation_generator = train_datagen.flow_from_directory(
  validation_dir,
  target_size = (150, 150),
  batch_size = 20,
  class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
history = model.fit_generator(
  train_generator,
  steps_per_epoch = 100,
  epochs = 5,
  validation_data = validation_generator,
  validation_steps = 50,
  verbose = 2
)

Epoch 1/5
 - 14s - loss: 0.7057 - acc: 0.5130 - val_loss: 0.6923 - val_acc: 0.5030
Epoch 2/5
 - 7s - loss: 0.6930 - acc: 0.5385 - val_loss: 0.6905 - val_acc: 0.5100
Epoch 3/5
 - 7s - loss: 0.6911 - acc: 0.5225 - val_loss: 0.6893 - val_acc: 0.5360
Epoch 4/5
 - 7s - loss: 0.6710 - acc: 0.5985 - val_loss: 0.6490 - val_acc: 0.6340
Epoch 5/5
 - 8s - loss: 0.6484 - acc: 0.6185 - val_loss: 0.6781 - val_acc: 0.5990
